# Puzzle

https://thefiddler.substack.com/p/the-randy-hall-problem

### This Week’s Fiddler

You are a producer on a game show hosted by Randy “Random” Hall (no relation to Monty Hall). The show has three doors labeled 1 through 3 from left to right, and behind them are various prizes.

Contestants pick one of the three doors at which to start, and then they press an electronic button many, many times in rapid succession. Each time they press the button, they either stay at their current door or move to an adjacent door. If they’re at door 2 and move to an adjacent door, that new door will be 1 or 3 with equal probability.

Randy has decided that when a contestant presses the button while at door 2, there should be a 20 percent chance they remain at door 2.

As the producer, you want the chances of a contestant ultimately winding up at each of the three doors to be nearly equal after many button presses. Otherwise, mathematicians will no doubt write you nasty letters complaining about how your show is rigged.

If a contestant presses the button while at door 1 (or door 3), what should the probability be that they remain at that door?

### This Week’s Extra Credit

Randy has an updated suggestion for how the button should behave at door 2. What hasn’t changed is that if a contestant at door 2 and moves to an adjacent door, that new door will be 1 or 3 with equal probability.

But this time, on the first, third, fifth, and other odd button presses, there’s a 20 percent the contestant remains at door 2 (if they happen to be there). On the second, fourth, sixth, and other even button presses, there’s a 50 percent chance the contest remains at door 2 (again, if they happen to be there).

Meanwhile, the button’s behavior at doors 1 and 3 should in no way depend on the number of times the button has been pressed.

As the producer, you want the chances of winding up at each of the three doors—after a large even number of button presses— to be nearly equal.

If a contestant presses the button while at door 1 (or door 3), what should the probability be that they remain at that door?

# Fiddler Solution

Took me a while to figure out how to approach this, but I finally realized that looking at the equilibrium condition was a good route in.

Let (x,y,z) be the probability distribution of being at doors 1,2 and 3 respectively at any given time. 
We start with (1,0,0), (0,1,0) or (0,0,1) and as the buttons are pressed, the probability distribution evolves.
In the long term equilibrium with the ideal setup, the distribution would (1/3, 1/3, 1/3).

Since the ideal setup + the equal distribution should be stable, the distribution should remain the same after one more step.
So, we can write out some equations and solve.

Let $c = 0.2$ be the probability of remaining when at the central door, and  $s$ be the probability of remaining when at the side doors.

The distribution after one step starting from the equal distribution would be $(1/3*s + 1/3*(1-c)/2, 1/3*c + 2*1/3*(1-s), 1/3*s + 1/3*(1-c)/2)$

Setting either the middle or the side term to 1/3 will allow us to solve for $s$.

$1/3 = 1/3*s + 1/3*(1-c)/2$

$1 = s + (1-c)/2$

$1 = s + 0.4$

$s = 0.6$

Now, it remains to show that starting from the various initial conditions, we will actually converge to the stable distribution. I'll use some code for that.

In [31]:
def fiddler_1step(dist, s=0.6, c=0.2):
    """
    Parameters:
    dist (list of float): Current distribution.
    s (float): side remain coefficient.
    c (float): center remain coefficient.

    Returns: Updated distribution.
    """
    (x, y, z) = dist
    new_x = s*x + ((1-c)/2)*y
    new_y = c*y + (1-s)*(x + z)
    new_z = s*z + ((1-c)/2)*y
    return (new_x, new_y, new_z)

def simulate_evolution(initial_dist, steps=100, step_fn=fiddler_1step):
    """
    Simulate the evolution of a distribution over a number of steps.

    Parameters:
    initial_dist (list of float): Initial distribution.
    steps (int): Number of evolution steps.
    step_fn (function): Function to compute the next step.
    """
    dist = initial_dist
    for i in range(steps):
        if i % 20 == 0 or i < 3 or (steps - 3) < i:  # Print every 20 steps and first/last 3 steps
            print(f"{i}: {dist}")
        dist = step_fn(dist)
        
    print(f"---")

simulate_evolution((1, 0, 0), step_fn=fiddler_1step)
simulate_evolution((0, 1, 0), step_fn=fiddler_1step)
simulate_evolution((0, 0, 1), step_fn=fiddler_1step)

0: (1, 0, 0)
1: (0.6, 0.4, 0.0)
2: (0.52, 0.32, 0.16000000000000003)
20: (0.3333516141255355, 0.33333333333332993, 0.33331505254113486)
40: (0.33333333400170817, 0.3333333333333335, 0.3333333326649587)
60: (0.3333333333333579, 0.3333333333333335, 0.33333333333330906)
80: (0.3333333333333335, 0.3333333333333335, 0.33333333333333337)
98: (0.3333333333333335, 0.3333333333333335, 0.33333333333333337)
99: (0.3333333333333335, 0.3333333333333335, 0.33333333333333337)
---
0: (0, 1, 0)
1: (0.4, 0.2, 0.4)
2: (0.32, 0.3600000000000001, 0.32)
20: (0.33333333333333, 0.3333333333333405, 0.33333333333333)
40: (0.3333333333333335, 0.3333333333333335, 0.3333333333333335)
60: (0.3333333333333335, 0.3333333333333335, 0.3333333333333335)
80: (0.3333333333333335, 0.3333333333333335, 0.3333333333333335)
98: (0.3333333333333335, 0.3333333333333335, 0.3333333333333335)
99: (0.3333333333333335, 0.3333333333333335, 0.3333333333333335)
---
0: (0, 0, 1)
1: (0.0, 0.4, 0.6)
2: (0.16000000000000003, 0.32, 0.52)
20:

It is clear that all 3 possible initial conditions quickly converge to the uniform distribution with the setting of s=0.6

**Fiddler Answer = 0.6**

# Extra Credit Solution

Now since the evolutionary steps alternate c=0.2 and c=0.5, we need to consider 2 steps applies to the equilibirum distribution.

Let's start with the distribution (1,1,1) even though it is not normalized (can be thought of as counts instead of probabilities).

The distribution after one step starting from the equal distribution would be $(s + (1-c)/2, c + 2*(1-s), s + (1-c)/2)$.

Plugging in c=0.2, we get 

$(s + 0.4, 0.2 + 2*(1-s), s + 0.4)$

$(s + 0.4, 2.2 - 2s, s + 0.4)$

Now, evolving this by one more step with c = 0.5, we get

$(s*(s + 0.4) + (2.2 - 2s)*0.25 , (2.2 - 2s)*0.5 + 2*(1-s)*(s+0.4), s*(s + 0.4) + (2.2 - 2s)*0.25)$

$(s^2 + 0.4s - 0.5s + 0.55, 1.1 - s - 2s^2 + 1.2s + 0.8 ,s^2 + 0.4s - 0.5s + 0.55)$

$(s^2 - 0.1s + 0.55, 1.9 + 0.2s - 2s^2, s^2 + - 0.1s + 0.55)$

Setting either the center or the side term to 1, we can get a quadratic and solve it.

$s^2 - 0.1s + 0.55 = 1$

$s^2 - 0.1s + 0.0025 = 0.4525$

$(s - 0.05)^2 = 0.4525$

$(s - 0.05) = 0.67268120235368551585815427310852 $ // Ignoring the negative solution

$s = 0.72268120235368551585815427310852 $

[Wolfram Alpha](https://www.wolframalpha.com/input?i=solve+1.9+%2B+0.2s+-+2s%5E2+%3D+1) tells me that this is $= \frac{1 + \sqrt{181}}{20}$

Let's verify the solution by running some code.

In [32]:
def extra_credit_2step(dist, s=0.72268120235368551585815427310852):
    """
    Parameters:
    dist (list of float): Current distribution.
    s (float): side remain coefficient.

    Returns:
    list of float: Updated distribution after two steps.
    """
    distA = dist
    distB = fiddler_1step(distA, c=0.2, s=s)
    distC = fiddler_1step(distB, c=0.5, s=s)
    return distC

simulate_evolution((1, 0, 0), step_fn=extra_credit_2step)
simulate_evolution((0, 1, 0), step_fn=extra_credit_2step)
simulate_evolution((0, 0, 1), step_fn=extra_credit_2step)

0: (1, 0, 0)
1: (0.5915978196469471, 0.33907248094147424, 0.06932969941157863)
2: (0.4697647347633281, 0.3332345198875293, 0.1970007453491427)
20: (0.3333344731640122, 0.3333333333333335, 0.33333219350265464)
40: (0.33333333333593185, 0.3333333333333334, 0.333333333330735)
60: (0.3333333333333335, 0.3333333333333335, 0.33333333333333337)
80: (0.3333333333333335, 0.3333333333333335, 0.33333333333333337)
98: (0.3333333333333335, 0.3333333333333335, 0.33333333333333337)
99: (0.3333333333333335, 0.3333333333333335, 0.33333333333333337)
---
0: (0, 1, 0)
1: (0.3390724809414742, 0.32185503811705163, 0.3390724809414742)
2: (0.3332345198875293, 0.33353096022494155, 0.3332345198875293)
20: (0.3333333333333336, 0.33333333333333365, 0.3333333333333336)
40: (0.3333333333333336, 0.33333333333333365, 0.3333333333333336)
60: (0.3333333333333336, 0.33333333333333365, 0.3333333333333336)
80: (0.3333333333333336, 0.33333333333333365, 0.3333333333333336)
98: (0.3333333333333336, 0.33333333333333365, 0.333

I noticed that it is quite sensitive to the exact value. Using something like 0.7226812 still converges, but not as fast and as clearly (subjectively speaking).

In any case, $s = 0.72268120235368551585815427310852 = \frac{1 + \sqrt{181}}{20}$ is the solution to the extra credit.